In [4]:
import pandas as pd
import numpy as np

In [5]:
myData = pd.read_csv('2021_10_20_products_series_labelled.csv')

In [6]:
myData

,omsid,product_name,description,brand,bullets,series_name
0,313296145,JAX Standard 8 in. Widespread 2-Handle Quarter...,Smooth modern lines come together with this po...,NaN,High pressure ceramic cartridge|Solid brass co...,Jax
1,313296138,JAX Standard 8 in. Widespread 2-Handle Quarter...,Smooth modern lines come together with this po...,NaN,"Solid brass construction|Supply lines, valves ...",Jax
2,313296149,JAX Standard 8 in. Widespread 2-Handle Quarter...,Smooth modern lines come together with this po...,NaN,Standard plumbing installation|ADA compliant|c...,Jax
3,313296158,JAX Standard 8 in. Widespread 2-Handle Quarter...,Smooth modern lines come together with this po...,NaN,ADA compliant|Solid brass construction|High pr...,Jax
4,315696655,AN-4500 and AN-4512 Aria Single Handle Single ...,NaN,NaN,Single-hole installation|cUPC certified|All mo...,Aria
...,...,...,...,...,...,...
29670,313341432,Colinet 2-Handle Wall Mount Bathroom Faucet Tr...,"Colinet, is a highly distinctive suite that pe...",.Proprietary Brand - Not In List,"Built on the MOEN M-PACT common valve system, ...",Colinet
29671,313341429,Colinet 2-Handle Wall Mount Bathroom Faucet Tr...,"Colinet, is a highly distinctive suite that pe...",.Proprietary Brand - Not In List,Brushed gold finish adds a touch of sophistica...,Colinet
29672,313341430,Colinet 2-Handle Wall Mount Bathroom Faucet Tr...,"Colinet, is a highly distinctive suite that pe...",.Proprietary Brand - Not In List,"Built on the MOEN M-pact common valve system, ...",Colinet
29673,313341434,Colinet 2-Handle Wall Mount Bathroom Faucet Tr...,"Colinet, is a highly distinctive suite that pe...",.Proprietary Brand - Not In List,Brushed nickel finish provides a lightly brush...,Colinet


In [204]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [167]:
train_contexts[1][207:226],len('singing and dancing')

('singing and dancing', 19)

In [168]:
train_answers[:10]

[{'text': 'in the late 1990s', 'answer_start': 269, 'answer_end': 286},
 {'text': 'singing and dancing', 'answer_start': 207, 'answer_end': 226},
 {'text': '2003', 'answer_start': 526, 'answer_end': 530},
 {'text': 'Houston, Texas', 'answer_start': 166, 'answer_end': 180},
 {'text': 'late 1990s', 'answer_start': 276, 'answer_end': 286},
 {'text': "Destiny's Child", 'answer_start': 320, 'answer_end': 335},
 {'text': 'Dangerously in Love', 'answer_start': 505, 'answer_end': 524},
 {'text': 'Mathew Knowles', 'answer_start': 360, 'answer_end': 374},
 {'text': 'late 1990s', 'answer_start': 276, 'answer_end': 286},
 {'text': 'lead singer', 'answer_start': 290, 'answer_end': 301}]

In [7]:
myData = myData.loc[:,['omsid','product_name','series_name']]

In [8]:
myData.loc[:,'answer_start'] = "N"
myData.loc[:,'answer_end'] = "N"

In [9]:
counter = 0
for i, row in myData.iterrows():
    if row.series_name in row.product_name:
        myData.loc[i,'answer_start'] = row.product_name.index(row.series_name)
        myData.loc[i,'answer_end'] = myData.loc[i,'answer_start'] + len(row.series_name)
        counter += 1
print(f"out of {myData.shape[0]}, {counter} are in the title")

out of 29675, 23391 are in the title


In [172]:
23391/29675

0.7882392586352148

In [10]:
myData_cleaned = myData.loc[myData.loc[:,'answer_start'] != "N", :]

In [11]:
myData_oos = myData.loc[myData.loc[:,'answer_start'] == "N", :]

In [12]:
myData_oos

,omsid,product_name,series_name,answer_start,answer_end
0,313296145,JAX Standard 8 in. Widespread 2-Handle Quarter...,Jax,N,N
1,313296138,JAX Standard 8 in. Widespread 2-Handle Quarter...,Jax,N,N
2,313296149,JAX Standard 8 in. Widespread 2-Handle Quarter...,Jax,N,N
3,313296158,JAX Standard 8 in. Widespread 2-Handle Quarter...,Jax,N,N
13,313026686,WALTZ 8 in. Widespread 2-Handle Lavatory Bathr...,Waltz,N,N
...,...,...,...,...,...
29660,305303412,1-Spray 1 in. Single Wall Mount Waterfall Hand...,Handheld Shower Heads,N,N
29663,314630664,Geneva 30 in. Farmhouse Apron Front Kitchen Si...,GENEVA,N,N
29664,314630667,Champlain 30 in. Farmhouse Apron Front Kitchen...,CHAMPLAIN,N,N
29665,314630665,Fortress 29 3/4 in. White Farmhouse Apron Fron...,ROUND APRON,N,N


In [173]:
from sklearn.model_selection import train_test_split

In [174]:
train, test = train_test_split(myData_cleaned, test_size=0.2)

In [67]:
test

[]

In [10]:
train_answers, train_contexts, train_questions = [], [], []
for i, row in train.iterrows():
    train_contexts.append(row.product_name)
    train_answers.append({'text': row.series_name, 'answer_start': row.answer_start, 'answer_end': row.answer_end})
    train_questions.append('What is the series name of this product?')
    
test_answers, test_contexts, test_questions = [], [], []
for i, row in test.iterrows():
    test_contexts.append(row.product_name)
    test_answers.append({'text': row.series_name, 'answer_start': row.answer_start, 'answer_end': row.answer_end})
    test_questions.append('What is the series name of this product?')
    

In [13]:
test_oos_answers, test_oos_contexts, test_oos_questions = [], [], []
for i, row in myData_oos.iterrows():
    test_oos_contexts.append(row.product_name)
    test_oos_answers.append({'text': row.series_name, 'answer_start': None, 'answer_end': None})
    test_oos_questions.append('What is the series name of this product?')

In [121]:
# from transformers import DistilBertTokenizerFast
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-uncased')
len(train_answers)


18712

In [76]:
ls roberta-base-squad2/

README.md           merges.txt         special_tokens_map.json
config.json         pytorch_model.bin  tokenizer_config.json
flax_model.msgpack  rust_model.ot      vocab.json


In [113]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True)

In [14]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
# train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(test_oos_contexts, test_oos_questions, truncation=True, padding=True)

In [15]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# add_token_positions(train_encodings, train_answers)
# add_token_positions(val_encodings, test_answers)

In [31]:
## add validation true vals...
val_encodings['true_aws'] = []
for i, row in myData_oos.iterrows():
    val_encodings['true_aws'].append(row.series_name)

In [33]:
val_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'true_aws'])

In [35]:

import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training and validation sets
#train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [37]:
from transformers import DistilBertForQuestionAnswering
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
# model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

In [19]:


# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(3):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 2: 100%|██████████| 1170/1170 [01:55<00:00, 10.14it/s, loss=0.00322] 


In [18]:
from transformers import pipeline

In [68]:
qa = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [77]:
question="What is the series name of this product?"
#qa(question=question, context=test.iloc[0,1])

In [43]:
del val_encodings['true_aws']

In [48]:
# switch model out of training mode
model.to(device)
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# initialize validation set data loader
val_loader = DataLoader(val_dataset, batch_size=16)
# initialize list to store accuracies
acc = []
# loop through batches
all_preds, all_truths = [],[]
for batch in val_loader:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        # we will use true positions for accuracy calc
        # start_true = batch['start_positions'].to(device)
        # end_true = batch['end_positions'].to(device)
        # truth_batch = batch['true_aws']
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull prediction tensors out and argmax to get predicted tokens
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        # calculate accuracy for both and append to accuracy list
        # acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        # acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
        pred_answer_batch = [ tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[x][start_pred[x]:end_pred[x]+1])) for x in range(input_ids.shape[0])]
        # true_answer_batch = [ tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[x][start_true[x]:end_true[x]+1])) for x in range(input_ids.shape[0])]
        all_preds.extend(pred_answer_batch)
        # all_truths.extend(truth_batch)
# calculate average accuracy in total
# acc = sum(acc)/len(acc)

ZeroDivisionError: division by zero

In [56]:
myData_oos.series_name.values

array(['Jax', 'Jax', 'Jax', ..., 'CHAMPLAIN', 'ROUND APRON',
       'DROP-IN COUNTER SINK PORCELAIN DOUBLE'], dtype=object)

In [65]:
counter = 0
for i, pred in enumerate(all_preds):
    if pred == myData_oos.series_name.values[i].lower():
        print(pred,"++++", myData_oos.series_name.values[i].lower())
        counter+=1
print(counter/len(all_preds))

jax ++++ jax
jax ++++ jax
jax ++++ jax
jax ++++ jax
waltz ++++ waltz
waltz ++++ waltz
waltz ++++ waltz
waltz ++++ waltz
waltz ++++ waltz
waltz ++++ waltz
myselectsmart ++++ myselectsmart
myselectsmart ++++ myselectsmart
eva ++++ eva
eva ++++ eva
eva ++++ eva
sto ++++ sto
sto ++++ sto
sto ++++ sto
sto ++++ sto
sto ++++ sto
sto ++++ sto
sto ++++ sto
sto ++++ sto
sto ++++ sto
aquaspec ++++ aquaspec
aquaspec ++++ aquaspec
aquasense ++++ aquasense
aquasense ++++ aquasense
aquasense ++++ aquasense
aquasense ++++ aquasense
aquasense ++++ aquasense
aquasense ++++ aquasense
aquasense ++++ aquasense
aquasense ++++ aquasense
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moore ++++ moore
moor

In [60]:
np.sum(list(map(lambda x:x.lower(),myData_oos.series_name.values)) == np.array(all_preds))/len(all_preds)

0.0549013367281986

In [176]:
true_answer_batch

['noah plus',
 'triton bowe cannock',
 'crosstown',
 'degas',
 'english country',
 'karsen ii',
 'diamond']

In [157]:
import numpy as np

np.sum(np.array(all_truths) == np.array(all_preds))/len(all_preds)

0.9805513998717674

In [160]:
model_path = "distilbert-custom/"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('distilbert-custom/tokenizer_config.json',
 'distilbert-custom/special_tokens_map.json',
 'distilbert-custom/vocab.txt',
 'distilbert-custom/added_tokens.json',
 'distilbert-custom/tokenizer.json')

In [20]:
##

model_path = 'distilbert-custom/'
model = DistilBertForQuestionAnswering.from_pretrained(model_path)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)


In [197]:
test.loc[:,'naive'] = test['product_name'].apply(lambda x: x.split()[0].lower())
test.loc[:,'series_name_lower'] = test['series_name'].apply(lambda x: x.lower())

In [201]:
np.sum(np.array(test.series_name_lower.values) == np.array(test.naive.values))/len(all_preds)

0.8179098097884163

In [202]:
test

,omsid,product_name,series_name,answer_start,answer_end,naive,series_name_lower
21277,310216155,Glacier Bay Drop-in/Undermount Granite Composi...,Glacier Bay,0,11,glacier,glacier bay
14261,301389122,IllumaTech 1000VA Preset Electronic Mark 10 Po...,IllumaTech,0,10,illumatech,illumatech
21655,315772211,Regina Double Handle Wall Mounted Bathroom Fau...,Regina,0,6,regina,regina
2278,313893380,Halsey Single Hole Single-Handle Bathroom Fauc...,Halsey,0,6,halsey,halsey
15620,206943633,Claymore Single Hole Single-Handle Bathroom Fa...,Claymore,0,8,claymore,claymore
...,...,...,...,...,...,...,...
9306,300316342,Viper Single-Handle Side Sprayer Kitchen Fauce...,Viper,0,5,viper,viper
10059,312859690,Graze Wall-Mount Pot Filler Kitchen Faucet in ...,Graze,0,5,graze,graze
11080,300277196,Riverby Undermount Cast Iron 25 in. 5-Hole Sin...,Riverby,0,7,riverby,riverby
958,308307507,Gibson 8 in. Widespread 2-Handle High-Arc Bath...,Gibson,0,6,gibson,gibson
